# Importing Modules

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, GlobalAveragePooling2D, Reshape

# Cloning Repo

In [3]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
Updating files:  11% (1570/13996)
Updating files:  12% (1680/13996)
Updating files:  13% (1820/13996)
Updating files:  14% (1960/13996)
Updating files:  15% (2100/13996)
Updating files:  16% (2240/13996)
Updating files:  17% (2380/13996)
Updating files:  18% (2520/13996)
Updating files:  19% (2660/13996)
Updating files:  20% (2800/13996)
Updating files:  21% (2940/13996)
Updating files:  22% (3080/13996)
Updating files:  22% (3136/13996)
Updating files:  23% (3220/13996)
Updating files:  24% (3360/13996)
Updating files:  25% (3499/13996)
Updating files:  26% (3639/13996)
Updating files:  27% (3779/13996)
Updating files:  28% (3919/13996)
Updating files:  29% (4059/13996)
Updating files:  30% (4199/13996)
Updating files:  31% (4339/13996)
Updating files:  31% (4462/13996)
Updating files:  32% (4479/13996)
Updating files:  33% (4619/13996)
Updating files:  34% (4759/13996)
Updating files:  35% (4899/13996)
Updating files:  36% (5039/13996)
Updating fi

# Creating appropriate structure for ImageDataGenerator

In [2]:
import os
os.path.relpath("C:/Users/ishan/Desktop/Coding Ninjas/Projects/Project 10_11 Week 31/facial_expressions/facial_expressions/data/legend.csv", start = os.curdir)

'facial_expressions\\data\\legend.csv'

In [3]:
import csv
data = {}
with open('facial_expressions\\data\\legend.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        key = row[2].lower()
        if key in data:
            data[key].append(row[1])
        else:
            data[key] = [row[1]]

In [4]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [26]:
os.mkdir('master_data')
os.mkdir('master_data\\training')
os.mkdir('master_data\\testing')

In [27]:
for emotion in emotion_list:
    os.mkdir(os.path.join('master_data\\training\\', emotion))
    os.mkdir(os.path.join('master_data\\testing\\', emotion))

In [32]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
    train_size = int(split_size * len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]
    for image in train_images:
        source = os.path.join('facial_expressions\\images\\', image)
        dest = os.path.join("master_data\\training\\", emotion, image)
        copyfile(source, dest)
    for image in test_images:
        source = os.path.join('facial_expressions\\images\\', image)
        dest = os.path.join("master_data\\testing\\", emotion, image)
        copyfile(source, dest)

# Creating Train and Test Generator

In [41]:
train_dir = 'master_data\\training\\'
test_dir = 'master_data\\testing\\'

# train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    rotation_range=20,    # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by 20% of the height
    shear_range=0.2,  # Apply shearing transformations
    zoom_range=0.2,   # Randomly zoom in/out by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode="nearest"  # Fill empty pixels created during transformations
)
train_generator = train_datagen.flow_from_directory(
                                                        train_dir,
                                                        target_size = (100, 100),
                                                        class_mode = 'categorical',
                                                        batch_size = 128
                                                    )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                        test_dir,
                                                        target_size = (100, 100),
                                                        class_mode = 'categorical',
                                                        batch_size = 128
                                                    )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


# Creating Model (Self) (Ignore this)

In [6]:
regularizer = tf.keras.regularizers.l2(l2=0.01)
self_model = tf.keras.models.Sequential([
    Conv2D(8, (3,3), activation = 'relu', input_shape = (100, 100, 3), kernel_regularizer=regularizer),
    MaxPooling2D(2, 2),
    Conv2D(16, (3,3), activation = 'relu', kernel_regularizer=regularizer),
    MaxPooling2D(2, 2),
    Conv2D(32, (3,3), activation = 'relu', kernel_regularizer=regularizer),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(units = 1024, activation = 'relu'),
    Dense(units = 8, activation = 'softmax')
])
self_model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['accuracy'])
self_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 8)         224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0

# Creating Model (Transfer Learning)

# Method 1: Custom Input Shape to ResNet50

In [36]:
#Get back the convolutional part of a VGG network trained on ImageNet
model_r50_conv = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)
model_r50_conv.trainable = False
model_r50_conv.summary()

#Create your own input format (here 100x100x3)
input = Input(shape=(100,100,3),name = 'image_input')

#Use the generated model 
output_r50_conv = model_r50_conv(input)

#Add the fully-connected layers
x = GlobalAveragePooling2D()(model_r50_conv.output)
x = Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = Dense(8, activation='softmax')(x)


##
# x = Flatten(name='flatten')(output_r50_conv)
# x = Dense(64, activation='relu', name='fc1')(x)
# x = Dense(8, activation='softmax', name='predictions')(x)

#Create your own model
my_model = tf.keras.models.Model(inputs=model_r50_conv.input, outputs=output)
# my_model = tf.keras.Model(input, x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_15[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_15[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [39]:
my_model.compile(loss = "categorical_crossentropy",
                    optimizer = Adam(learning_rate = 1e-4),
                    metrics=["accuracy"])

In [42]:
checkpoint = ModelCheckpoint("resnet50_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=2, verbose=1, mode='auto')
my_model.fit(train_generator, epochs= 10, validation_data= test_generator, callbacks=[checkpoint,early])
my_model.save_weights("resnet50_1.h5")

Epoch 1/10
86/86 [==============================] - 49s 566ms/step - loss: 1.0582 - accuracy: 0.5018 - val_loss: 1.0412 - val_accuracy: 0.5011

Epoch 00001: val_accuracy improved from -inf to 0.50109, saving model to resnet50_1.h5
Epoch 2/10
86/86 [==============================] - 59s 687ms/step - loss: 1.0583 - accuracy: 0.5018 - val_loss: 1.0409 - val_accuracy: 0.5011

Epoch 00002: val_accuracy did not improve from 0.50109
Epoch 3/10
86/86 [==============================] - 66s 768ms/step - loss: 1.0602 - accuracy: 0.5018 - val_loss: 1.0404 - val_accuracy: 0.5011

Epoch 00003: val_accuracy did not improve from 0.50109
Epoch 00003: early stopping


# Alternative Approach (VGG16)

In [43]:
from tensorflow.keras.models import Model

# Load VGG16 without the top layer
base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(100, 100, 3), pooling='avg')
base_model.trainable = False

# Input layer
inputs = Input(shape=(100, 100, 3))

# Add the pre-trained model
x = base_model(inputs, training=False)

# Additional layers
# x = Dense(1024, activation='relu')(x)
# x = Reshape((32, 32, 1))(x)
# x = Conv2D(8, (3, 3), activation='relu')(x)
# x = MaxPooling2D((2, 2))(x)
# x = Conv2D(16, (3, 3), activation='relu')(x)
# x = MaxPooling2D((2, 2))(x)
# x = Flatten()(x)
x = Dense(1024, activation='relu')(x)

# Output layer
outputs = Dense(8, activation='softmax')(x)

# Final model
final_model = Model(inputs, outputs)

# Summary to verify
final_model.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 100, 100, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 8200      
Total params: 15,248,200
Trainable params: 533,512
Non-trainable params: 14,714,688
_________________________________________________________________


In [44]:
final_model.compile(loss = "categorical_crossentropy",
                    optimizer = Adam(learning_rate = 0.01),
                    metrics=["accuracy"])

In [45]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=2, verbose=1, mode='auto')
final_model.fit(train_generator, epochs= 10, validation_data= test_generator, callbacks=[checkpoint,early])
final_model.save_weights("vgg16_1.h5")

Epoch 1/10
86/86 [==============================] - 62s 639ms/step - loss: 1.1922 - accuracy: 0.5707 - val_loss: 0.9770 - val_accuracy: 0.5744

Epoch 00001: val_accuracy improved from -inf to 0.57440, saving model to vgg16_1.h5
Epoch 2/10
86/86 [==============================] - 58s 674ms/step - loss: 0.8255 - accuracy: 0.6679 - val_loss: 1.1925 - val_accuracy: 0.5711

Epoch 00002: val_accuracy did not improve from 0.57440
Epoch 3/10
86/86 [==============================] - 60s 698ms/step - loss: 0.8088 - accuracy: 0.6807 - val_loss: 1.3126 - val_accuracy: 0.5416

Epoch 00003: val_accuracy did not improve from 0.57440
Epoch 00003: early stopping


# Alternate Approach, need to change input from (100,100,3) to (224,224,3)

In [30]:
base_model = tf.keras.applications.VGG16(
    include_top=True,
    weights="imagenet"
#     input_shape = (100, 100, 3)
)

for layers in (base_model.layers)[:19]:
    print(layers)
    layers.trainable = False
# base_model.trainable = False

ResourceExhaustedError: OOM when allocating tensor with shape[25088,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [5]:
backup = base_model

In [9]:
X= backup.layers[-2].output
predictions = Dense(8, activation="softmax")(X)
model_final = tf.keras.Model(backup.input, predictions)

In [13]:
model_final.compile(loss = "categorical_crossentropy",
                    optimizer = SGD(learning_rate=0.0001, momentum=0.9),
                    metrics=["accuracy"])
model_final.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [20]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=2, verbose=1, mode='auto')
model_final.fit(train_generator, epochs= 10, validation_data= test_generator, callbacks=[checkpoint,early])
model_final.save_weights("vgg16_1.h5")

Epoch 1/10


InvalidArgumentError:  Input to reshape is a tensor with 589824 values, but the requested shape requires a multiple of 25088
	 [[node model/flatten/Reshape (defined at <ipython-input-18-d17f433a2e15>:3) ]] [Op:__inference_train_function_1510]

Function call stack:
train_function


In [87]:
es = EarlyStopping(monitor = 'val_accuracy', patience = 2, min_delta = 0.01)

In [92]:
model.fit(train_generator, epochs = 10, verbose = 1, validation_data = test_generator, callbacks = [es])

Epoch 1/10
86/86 [==============================] - 18s 201ms/step - loss: 1.2805 - accuracy: 0.4956 - val_loss: 1.1089 - val_accuracy: 0.5077
Epoch 2/10
86/86 [==============================] - 17s 200ms/step - loss: 0.9757 - accuracy: 0.6435 - val_loss: 0.9319 - val_accuracy: 0.6834
Epoch 3/10
86/86 [==============================] - 17s 199ms/step - loss: 0.7525 - accuracy: 0.7584 - val_loss: 0.9922 - val_accuracy: 0.6627
Epoch 4/10
86/86 [==============================] - 17s 203ms/step - loss: 0.6889 - accuracy: 0.7729 - val_loss: 1.0983 - val_accuracy: 0.6462
